In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import deepRD
import deepRD.tools.analysisTools as analysisTools
from scipy.stats import wasserstein_distance

In [ ]:
def read_numpy_arrays_from_file(file_path):
    arrays = []
    with open(file_path, 'r') as file:
        contents = file.read()

    # Split the file content based on square brackets [ ]
    array_strings = contents.split('[')[1:]

    # Convert each array string back to a numpy array and store it
    for array_string in array_strings:
        array = np.fromstring(array_string.replace(']', ''), sep=' ')
        arrays.append(array)

    return arrays

In [ ]:
# Load parameters from parameters file
simID = '0004'
localDataDirectory = os.environ.get('DATA') + 'openSystems/classicSmoluchowski_' + simID
parameterDictionary = analysisTools.readParameters(localDataDirectory + "/parameters_" + simID)
# Parameters for loading continuous trajectories from files (from original simulation)
numSimulations = parameterDictionary['numSimulations']
D = parameterDictionary['D'] 
dt = parameterDictionary['dt']
stride = parameterDictionary['stride']
tfinal = parameterDictionary['tfinal']
kappa = parameterDictionary['kappa'] 
sigma = parameterDictionary['sigma']
R = parameterDictionary['R']
cR = parameterDictionary['cR']
equilibrationSteps = parameterDictionary['equilibrationSteps']

In [ ]:
# Read files
file_path = localDataDirectory + '/classicSmol_nsims' + str(numSimulations) + '.xyz'
distancesArrays = read_numpy_arrays_from_file(file_path)

In [ ]:
print(len(distancesArrays),cR)

In [ ]:
# Calculate average concentration with bootstrapping stdDevs
nbins = 30
#numSimulations = 1500
# Calculate values of middle points of histogram
hist, bins = np.histogram(distancesArrays[0], bins=nbins, range=(sigma,R))
rvals = np.zeros(len(bins)-1)
rvolumes = np.zeros(len(bins)-1)
binsize = bins[1]-bins[0]
for i in range(len(bins)-1):
    rvals[i] = (bins[i+1] + bins[i])/2.0
    rvolumes[i] = 4*np.pi*(bins[i+1]**3 - bins[i]**3)/3.0 # Approx alternative: 4*np.pi* rvals[i]**2*(bins[i+1]-bins[i])
    #rvolumes[i] = 4*np.pi* rvals[i]**2*(bins[i+1]-bins[i])   
# Calculate average concentration
concentrationHistogram = np.zeros(len(bins)-1)
stdDevs = np.zeros(len(bins)-1)
concHist = [None]*numSimulations
for i in range(numSimulations):
    hist, bins = np.histogram(distancesArrays[i], bins=nbins, range=(sigma,R))
    concHist[i] = hist/rvolumes
for j in range(len(concentrationHistogram)):
    concentrationHistogram[j] = np.mean(np.array(concHist)[:,j])
    stdDevs[j] = np.std(np.array(concHist)[:,j])

In [ ]:
fig, ax = plt.subplots()
bw=0.1
fsize = 15
# Calculate analytical solution
#kappa = 10.0
#sigma = 1.0 
#R = 10.0
#cR = 1.0
#rvals = np.arange(sigma,R,0.1)
kS = 4*np.pi*sigma*D
analyticCKRinf = cR*(1 - kappa*sigma/(kS + kappa)*(1/rvals))
analyticCK = cR*(1 - kappa*sigma/(kS + kappa*(1-sigma/R))*(1.0/rvals - 1.0/R))


ax.plot(rvals, analyticCK,'-k', linewidth =1, label='benchmark')
ax.plot(rvals, concentrationHistogram,'--b', linewidth = 2, label='simulation (n='+ str(numSimulations)+')')
ax.fill_between(rvals, concentrationHistogram-stdDevs, concentrationHistogram+stdDevs, alpha=0.1, 
                edgecolor='#1B2ACC', facecolor='#089FFF', linewidth=2, antialiased=True)
#ax.plot(rvals, analyticCKRinf,'-r', lw =3, label='benchmark Rinf')

ax.set_xlim((sigma, R))
ax.set_ylim((0, None))
ax.set_xlabel('radial distance',fontsize=fsize)
ax.set_ylabel('Concentration',fontsize=fsize)
ax.tick_params(axis='x', labelsize=fsize)
ax.tick_params(axis='y', labelsize=fsize)



ax.legend(bbox_to_anchor=(0.6, 0., 0.4, 0.3), framealpha=1.0,fontsize=fsize)

plt.tight_layout()

## Plots for paper

In [ ]:
# Calculate average concentration
nSimsList = [50,500,5000]
concentrationHistograms=[None]*len(nSimsList)
stdDevs = [None]*len(nSimsList)
for k, nsims in enumerate(nSimsList):
    concentrationHistograms[k] = np.zeros(nbins)
    stdDevs[k] = np.zeros(nbins)
    concHist = [None]*nsims
    for i in range(nsims):
        hist, bins = np.histogram(distancesArrays[i], bins=nbins, range=(sigma,R))
        concHist[i] = hist/rvolumes
    for j in range(len(concentrationHistograms[k])):
        concentrationHistograms[k][j] = np.mean(np.array(concHist)[:,j])
        stdDevs[k][j] = np.std(np.array(concHist)[:,j])

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(14,4), sharey=True, squeeze= True)
bw=0.1
fsize = 15

#Analytical solution
#cR= 1.0
kS = 4*np.pi*sigma*D
analyticCKRinf = cR*(1 - kappa*sigma/(kS + kappa)*(1/rvals))
analyticCK = cR*(1 - kappa*sigma/(kS + kappa*(1-sigma/R))*(1.0/rvals - 1.0/R))

for k, nsims in enumerate(nSimsList):
    ax[k].plot(rvals, analyticCK,'-k', linewidth =1, label='benchmark')
    ax[k].plot(rvals, concentrationHistograms[k],'--b', linewidth = 2, label='simulation (n='+ str(nsims) + ')')
    ax[k].fill_between(rvals, concentrationHistograms[k]-stdDevs[k], concentrationHistograms[k]+stdDevs[k], alpha=0.1, 
                    edgecolor='#1B2ACC', facecolor='#089FFF', linewidth=2, antialiased=True)
    #ax.plot(rvals, analyticCKRinf,'-r', lw =3, label='benchmark Rinf')

    #ax[k].set_xlim((sigma+binsize, R-binsize))
    ax[k].set_xlim((sigma, R))
    #ax[k].set_ylim((0, 1.75))
    ax[k].set_ylim((0, 1.25))
    #ax[k].set_ylim((0, 0.75))
    ax[k].set_xlabel('r',fontsize=fsize)
    ax[k].tick_params(axis='x', labelsize=fsize)
    ax[k].tick_params(axis='y', labelsize=fsize)
    ax[k].legend(bbox_to_anchor=(0.6, 0., 0.4, 0.3), framealpha=1.0,fontsize=fsize)


ax[0].set_ylabel('Concentration',fontsize=fsize)

plt.tight_layout()
plt.savefig('Smoluchowski_plots_cR=' + str(cR) + '.pdf')

### Large number convergence plot

In [ ]:
simIDs = ['0005','0004','0006'] 
cRList = [None]*len(simIDs)
distancesArraysList = [None]*len(simIDs)
for i, ID in enumerate(simIDs):
    localDataDirectory = os.environ.get('DATA') + 'openSystems/classicSmoluchowski_' + ID
    parameterDictionary = analysisTools.readParameters(localDataDirectory + "/parameters_" + ID)
    file_path = localDataDirectory + '/classicSmol_nsims' + str(numSimulations) + '.xyz'
    distancesArraysList[i] = read_numpy_arrays_from_file(file_path)
    cRList[i] = parameterDictionary['cR']
    if i==0: # Assumes all other parameters are the same for all simIDs
        # Parameters for loading continuous trajectories from files (from original simulation)
        numSimulations = parameterDictionary['numSimulations']
        D = parameterDictionary['D'] 
        dt = parameterDictionary['dt']
        stride = parameterDictionary['stride']
        tfinal = parameterDictionary['tfinal']
        kappa = parameterDictionary['kappa'] 
        sigma = parameterDictionary['sigma']
        R = parameterDictionary['R']
        equilibrationSteps = parameterDictionary['equilibrationSteps']

In [ ]:
# Calculate average concentration
concentrationHistograms=[None]*len(simIDs)
stdDevs = [None]*len(simIDs)
for k, ID in enumerate(simIDs):
    concentrationHistograms[k] = np.zeros(nbins)
    stdDevs[k] = np.zeros(nbins)
    concHist = [None]*numSimulations
    for i in range(numSimulations):
        hist, bins = np.histogram(distancesArraysList[k][i], bins=nbins, range=(sigma,R))
        concHist[i] = hist/rvolumes
    for j in range(nbins):
        concentrationHistograms[k][j] = np.mean(np.array(concHist)[:,j])
        stdDevs[k][j] = np.std(np.array(concHist)[:,j])

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
col = ['darkorange','firebrick', 'cornflowerblue']
col = ['cornflowerblue','cornflowerblue', 'cornflowerblue']
col = ['#089FFF']*3
coledge = ['#1B2ACC']*3
#col = ['k','k', 'k']
#coledge = ['k', 'k', 'k']
linew = [2,2,2]
alphaL = [0.07,0.14, 0.25]
ax.plot(rvals, analyticCK,'-k', linewidth =2)
for k, ID in enumerate(simIDs):
    cR = cRList[k]
    #ax.plot(rvals, concentrationHistograms[k]/cR, '--', color=col[k],linewidth = linew[k], label=r'($c_R=$'+ str(cR) + ')')
    ax.fill_between(rvals, (concentrationHistograms[k]-stdDevs[k])/cR, (concentrationHistograms[k]+stdDevs[k])/cR, alpha=1.0, 
                    edgecolor='w', facecolor='w', linewidth=2, antialiased=True)
    ax.fill_between(rvals, (concentrationHistograms[k]-stdDevs[k])/cR, (concentrationHistograms[k]+stdDevs[k])/cR, linestyle='-',
                    alpha=alphaL[k], edgecolor=coledge[k], facecolor=col[k], linewidth=2, antialiased=True, label=r'$c_R=$'+ str(cR))
    #ax.plot(rvals, analyticCKRinf,'-r', lw =3, label='benchmark Rinf')

    #ax[k].set_xlim((sigma+binsize, R-binsize))
    ax.set_xlim((sigma, R))
    ax.set_ylim((0, 1.4))
    ax.set_xlabel('r',fontsize=fsize)
    ax.tick_params(axis='x', labelsize=fsize)
    ax.tick_params(axis='y', labelsize=fsize)
    


ax.set_ylabel(r'Concentration$/c_R$',fontsize=fsize)
ax.legend(framealpha=1.0, fontsize=fsize, loc='lower right')
plt.tight_layout()
plt.savefig('Smoluchowski_LLN.pdf')

### Error plots as function of timestep and at partially absorbing boundary

In [ ]:
# Load files
#numdts = 5
#simIDs = ['0011','0012','0013','0014','0017','0021','0022','0023','0024','0027']
numdts = 7
simIDs = ['0011','0012','0013','0016','0017','0018','0014'] 
#simIDs = ['0011','0012','0013','0016','0017','0018','0014','0021','0022','0023','0026','0027','0028','0024']
#numdts = 6
#simIDs = ['0011','0012','0013','0017','0018','0014']

#numdts = 5
#simIDs = ['0011','0012','0013','0016','0017','0014'] 

#numdts = 4
#simIDs = ['0011','0012','0013','0014','0021','0022','0023','0024'] 
numSimulations = 1000
distancesArraysList = [None]*len(simIDs)
dtList = [None]*len(simIDs)
for i, ID in enumerate(simIDs):
    localDataDirectory = os.environ.get('DATA') + 'openSystems/classicSmoluchowski_' + ID
    parameterDictionary = analysisTools.readParameters(localDataDirectory + "/parameters_" + ID)
    file_path = localDataDirectory + '/classicSmol_nsims' + str(numSimulations) + '.xyz'
    distancesArraysList[i] = read_numpy_arrays_from_file(file_path)
    dtList[i] = parameterDictionary['dt']
    if i==0: # Assumes all other parameters are the same for all simIDs
        # Parameters for loading continuous trajectories from files (from original simulation)
        numSimulations = parameterDictionary['numSimulations']
        D = parameterDictionary['D'] 
        stride = parameterDictionary['stride']
        tfinal = parameterDictionary['tfinal']
        kappa = parameterDictionary['kappa'] 
        sigma = parameterDictionary['sigma']
        R = parameterDictionary['R']
        cR = parameterDictionary['cR']
        equilibrationSteps = parameterDictionary['equilibrationSteps']

In [ ]:
numSimulations = 700

In [ ]:
# Choose histogram Calculate values of middle points of histogram
nbins = 30
hist, bins = np.histogram(distancesArraysList[0][0], bins=nbins, range=(sigma,R))
rvals = np.zeros(len(bins)-1)
rvolumes = np.zeros(len(bins)-1)
binsize = bins[1]-bins[0]
for i in range(len(bins)-1):
    rvals[i] = (bins[i+1] + bins[i])/2.0
    rvolumes[i] = 4*np.pi*(bins[i+1]**3 - bins[i]**3)/3.0 # Approx alternative: 4*np.pi* rvals[i]**2*(bins[i+1]-bins[i])

#Analytical solution
kS = 4*np.pi*sigma*D
analyticCKRinf = cR*(1 - kappa*sigma/(kS + kappa)*(1/rvals))
analyticCK = cR*(1 - kappa*sigma/(kS + kappa*(1-sigma/R))*(1.0/rvals - 1.0/R))

In [ ]:
# Calculate average concentration and errors
concentrationHistograms=[None]*len(simIDs)
#meanErrors = [None]*len(simIDs)
#errorsStdDevs = [None]*len(simIDs)
#numBootstrapSamples = 250
for k, ID in enumerate(simIDs):
    concentrationHistograms[k] = np.zeros(nbins)
    concHist = [None]*numSimulations
    for i in range(numSimulations):
        hist, bins = np.histogram(distancesArraysList[k][i], bins=nbins, range=(sigma,R))
        concHist[i] = hist/rvolumes
    for j in range(nbins):
        concentrationHistograms[k][j] = np.mean(np.array(concHist)[:,j])
        
    
    ## Calculate boostrapped samples for error bars (averages histogram over numValues randomly selected trajs)
    #values = np.zeros([numBootstrapSamples,nbins])
    #numvalues = 25
    #for i in range(numBootstrapSamples):
    #    bootStrappedIndexes = np.random.choice(np.arange(0,len(concHist)), size = numvalues, replace = True)
    #    bootstrapSample = [concHist[index] for index in bootStrappedIndexes]
    #    for j in range(nbins):
    #        values[i,j] = np.mean(np.array(bootstrapSample)[:,j]) 
    #errorList = np.linalg.norm(np.array(values) - analyticCK, ord = 2, axis=1)
    #meanErrors[k], errorsStdDevs[k] = analysisTools.bootstrapping(errorList,numBootstrapSamples)

In [ ]:
# Extract and plot last values at partially absorbing boundary
#analyticVal = analyticCK[0]
#analyticVal = (analyticCK[0]+analyticCK[1]+analyticCK[2])
errorList = [None]*len(simIDs)
errorListBC = [None]*len(simIDs)
numpointsBC = 2
#errorList2 = [None]*len(simIDs)
#stdDevListMinus = [None]*len(simIDs)
#stdDevListPlus = [None]*len(simIDs)
for k, ID in enumerate(simIDs):
#    #numericalVal = concentrationHistograms[k][0]
#    #numericalVal = (concentrationHistograms[k][0] + concentrationHistograms[k][1]+ concentrationHistograms[k][2])/3.0
#    #errorList[k] = np.abs(numericalVal - analyticVal)
     #errorList[k]=wasserstein_distance(analyticCK,concentrationHistograms[k])
     errorList[k]=np.linalg.norm(analyticCK-concentrationHistograms[k])
     errorListBC[k] = np.linalg.norm(concentrationHistograms[k][0:numpointsBC] - analyticCK[0:numpointsBC])



fig, ax = plt.subplots(1,1, figsize=(5,3.5))    
#ax.plot(dtList[0:numdts],dtList[0:numdts], 'k--')
#ax.plot(dtList[0:numdts], meanErrors[0:numdts], 'k--', alpha = 0.3, lw=0.5)
#ax.errorbar(dtList[0:numdts], meanErrors[0:numdts], yerr=errorsStdDevs[0:numdts], fmt ='kd', label="First order")
#ax.plot(dtList[numdts:], meanErrors[numdts:], 'k--', alpha = 0.3, lw=0.5)
#ax.errorbar(dtList[numdts:], meanErrors[numdts:], yerr=errorsStdDevs[numdts:], fmt ='rx', label="Second order")

ax.plot(dtList[0:numdts], errorList[0:numdts], 'k--', alpha = 0.3, lw=0.5)
ax.plot(dtList[0:numdts], errorList[0:numdts], 'kd', label=r'$L^2$-error between'+ '\n full solutions') #label="First order")

ax.plot(dtList[0:numdts], errorListBC[0:numdts], '--', color='royalblue',  alpha = 0.3, lw=0.5)
ax.plot(dtList[0:numdts], errorListBC[0:numdts], 'P', color='royalblue', label=r'$L^2$-error (at $r=1$)') #label="First order")

#ax.plot(dtList[0:numdts], errorList2[0:numdts], 'r--', alpha = 0.3, lw=0.5)
#ax.plot(dtList[0:numdts], errorList2[0:numdts], 'rx', label=r'Wasserstein distance') #label="First order")
#ax.plot(dtList[numdts:], errorList[numdts:], 'kx',label="Second order")
ax.set_xlabel(r'log of simulation timestep ($log(\delta t)$)',fontsize=fsize)
ax.set_xscale('log')
ax.set_xlim([0.1,0.0005])
#ax.set_yscale('log')
#ax.set_xlim([0.055,-0.003])
#ax.set_ylim([0.1,0.25])
ax.tick_params(axis='x', labelsize=fsize)
ax.tick_params(axis='y', labelsize=fsize)
ax.legend(framealpha=1.0, fontsize=fsize)
ax.grid(linestyle = '--',alpha=0.3)
plt.tight_layout()
plt.savefig('Error_partially_absorbingBC_timestep.pdf')

### Error plots for reservoir

In [ ]:
# Load files
simIDs = ['0031','0032','0033','0034','0035','0036','0037','0038','0039','0040','0041'] 
numSimulations = 1000
distancesArraysList = [None]*len(simIDs)
tauStepsList = [None]*len(simIDs)
for i, ID in enumerate(simIDs):
    localDataDirectory = os.environ.get('DATA') + 'openSystems/classicSmoluchowski_' + ID
    parameterDictionary = analysisTools.readParameters(localDataDirectory + "/parameters_" + ID)
    file_path = localDataDirectory + '/classicSmol_nsims' + str(numSimulations) + '.xyz'
    distancesArraysList[i] = read_numpy_arrays_from_file(file_path)
    tauStepsList[i] = parameterDictionary['tauleapSubsteps']
    if i==0: # Assumes all other parameters are the same for all simIDs
        # Parameters for loading continuous trajectories from files (from original simulation)
        numSimulations = parameterDictionary['numSimulations']
        D = parameterDictionary['D'] 
        dt = parameterDictionary['dt']
        stride = parameterDictionary['stride']
        tfinal = parameterDictionary['tfinal']
        kappa = parameterDictionary['kappa'] 
        sigma = parameterDictionary['sigma']
        R = parameterDictionary['R']
        cR = parameterDictionary['cR']
        equilibrationSteps = parameterDictionary['equilibrationSteps']

In [ ]:
numSimulations=800

In [ ]:
# Choose histogram Calculate values of middle points of histogram
nbins = 30
hist, bins = np.histogram(distancesArraysList[0][0], bins=nbins, range=(sigma,R))
rvals = np.zeros(len(bins)-1)
rvolumes = np.zeros(len(bins)-1)
binsize = bins[1]-bins[0]
for i in range(len(bins)-1):
    rvals[i] = (bins[i+1] + bins[i])/2.0
    rvolumes[i] = 4*np.pi*(bins[i+1]**3 - bins[i]**3)/3.0 # Approx alternative: 4*np.pi* rvals[i]**2*(bins[i+1]-bins[i])
#Analytical solution
kS = 4*np.pi*sigma*D
analyticCKRinf = cR*(1 - kappa*sigma/(kS + kappa)*(1/rvals))
analyticCK = cR*(1 - kappa*sigma/(kS + kappa*(1-sigma/R))*(1.0/rvals - 1.0/R))

In [ ]:
# Calculate average concentration
concentrationHistograms=[None]*len(simIDs)
stdDevs = [None]*len(simIDs)
for k, ID in enumerate(simIDs):
    concentrationHistograms[k] = np.zeros(nbins)
    stdDevs[k] = np.zeros(nbins)
    concHist = [None]*numSimulations
    for i in range(numSimulations):
        hist, bins = np.histogram(distancesArraysList[k][i], bins=nbins, range=(sigma,R))
        concHist[i] = hist/rvolumes
    for j in range(len(concentrationHistograms[k])):
        concentrationHistograms[k][j] = np.mean(np.array(concHist)[:,j])
        stdDevs[k][j] = np.std(np.array(concHist)[:,j])

In [ ]:
# Extract and plot last values at partially absorbing boundary
#analyticVal = analyticCK[-1]
#analyticVal = (analyticCK[-1] + analyticCK[-2])/2.0
errorListTau = [None]*len(simIDs)
errorListTauBC = [None]*len(simIDs)
numpointsBC = 2
for k, ID in enumerate(simIDs):
    #numericalVal = concentrationHistograms[k][-1]
    #numericalVal = (concentrationHistograms[k][-1] + concentrationHistograms[k][-2])/2.0
    #errorListTau[k] = np.abs(numericalVal - analyticVal)
    #errorListTau[k] = numericalVal - analyticVal 
    #errorListTau[k]=wasserstein_distance(analyticCK,concentrationHistograms[k])
    errorListTau[k]=np.linalg.norm(analyticCK-concentrationHistograms[k])
    errorListTauBC[k] = np.linalg.norm(np.flip(concentrationHistograms[k])[0:numpointsBC] - np.flip(analyticCK)[0:numpointsBC])

fig, ax = plt.subplots(1,1, figsize=(5,3.5))    
ax.plot(tauStepsList, errorListTau, 'k--', alpha = 0.3, lw=0.5)
ax.plot(tauStepsList, errorListTau, 'kd', label=r'$L^2$-error between'+ '\n full solutions') #,label=r"$\tau-$leap error")

ax.plot(tauStepsList, errorListTauBC, '--', color='royalblue', alpha = 0.3, lw=0.5)
ax.plot(tauStepsList, errorListTauBC, 'P', color='royalblue', label=r'$L^2$-error (at $r=R$)') #label="First order")

ax.set_xlabel(r'$\tau$-leap substeps at reservoir interface',fontsize=fsize)
#ax.set_xscale('log')
#ax.set_xlim([0.0005,0.1])
#ax.set_yscale('log')
ax.tick_params(axis='x', labelsize=fsize)
ax.tick_params(axis='y', labelsize=fsize)
ax.legend(framealpha=1.0,fontsize=fsize)
ax.grid(linestyle = '--',alpha=0.3)
plt.tight_layout()
plt.savefig('Error_reservoir_numTauLeapSubsteps.pdf')

## Old plots

In [ ]:
fig, ax = plt.subplots()
bw=0.1
concDensity = stats.kde.gaussian_kde(distancesArrays[0],bw)

distance = np.arange(sigma, R, 0.02)
#ax.plot(distance, concDensity(distance), '-k') # label='benchmark')
ax.plot(distance, concDensity(distance)/(4*np.pi*distance**2),'--k') # label='benchmark')

ax.set_xlim((sigma, R))
ax.set_ylim((0, None))
ax.set_xlabel('radial distance')
ax.set_ylabel('Concentration')

#ax.legend(bbox_to_anchor=(0.6, 0., 0.5, 1.0), framealpha=1.0)

plt.tight_layout()